# A Hubble Source Catalog (HSC) Use Case

- [Example #5: Using the Discovery Portal and CasJobs to search for Outlier Objects in the HSC][1]
  - (White dwarfs in the Globular Cluster M4)

![Outlier_fig0][2]

  [1]: https://archive.stsci.edu/hst/hsc/help/use_case_5_v2.html
  [2]: screenshots/Outlier_fig0.png

<span style="color:red;">GOAL</span>: This tutorial shows you how to use the [MAST Discovery Portal][1] and [CasJobs][2] to make a Color-Magnitude Diagram to search for outliers. In the process, you will learn how to download a table from CasJobs, and upload it into the Discovery Portal.

<span style="color:red;">SCIENCE CASE</span>: The science case is searching for white dwarfs (i.e. in the Globular Cluster M4; [Richer et al. 1997, ApJ 484, 741][3]).


  [1]: http://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html
  [2]: http://mastweb.stsci.edu/hcasjobs/
  [3]: http://adsabs.harvard.edu/abs/1997ApJ...484..741R.html

In [ ]:
import astropy
from astropy.coordinates import SkyCoord
import time
import sys
import os
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from scipy.stats import linregress
from sklearn.linear_model import LinearRegression

from PIL import Image
from io import BytesIO

from astropy.table import Table, join
from astropy.io import ascii

# set width for pprint
astropy.conf.max_width = 150

In [ ]:
# import hsc catalog search utilites and image search utilites from local py files
from search_utils import *
from image_utils import *

In [ ]:
pd.set_option('display.max_columns', 700)
pd.set_option('display.max_rows', 400)
pd.set_option('display.min_rows', 10)
pd.set_option('display.expand_frame_repr', True)

## <span style="color:red;">Step 1</span> - Determine which object(s) you wish to study

Go to the [Hubble Legacy Archive][1] and input the position (<span style="color:blue;">blue</span>) of your candidate object (in this case the globular cluster M4). Go to the Footprints tab (<span style="color:green;">green</span>) to see if the HST coverage is sufficient for your project. In this case, there are many imaging observations available, so this is a good object for study.

_2022 March 19: The HLA footprint viewer no longer works because it relies on Flash. If you need footprints, we recommend using the MAST Portal. Note that the Portal also gives access to the new Hubble Advanced Products (HAP), which include more recent observations. **The HLA contains no observations acquired after 2017 October 1**._

![Outlier_discovery1][2]

In the Inventory tab, scroll over to the PropID field and click on the ID (for example proposal 12311 (<span style="color:blue;">blue</span>)). This will take you to the proposal page where you can see the abstract for the proposal, as well as papers that use this dataset.

![Outlier_discovery2][3]


  [1]: http://hla.stsci.edu/
  [2]: screenshots/Outlier_discovery1.png
  [3]: screenshots/Outlier_discovery2.png

## <span style="color:red;">Step 2</span> - Go to the [MAST Discovery Portal][1].

Use the pull down menu under Select Collection to choose the HSC (<span style="color:blue;">blue</span>). Enter (you can just cut and paste if you like) the coordinates and search radius (i.e. 16:23:35.4 -26:31:31.9 r=200s) in the Search box (<span style="color:green;">green</span>). Perform the search by just hitting a carriage return or by clicking on the  icon (<span style="color:orange;">orange</span>). The results are displayed in the List of Objects, while the AstroView window shows the objects against the DSS image. The left column is a series of Filters that can be used to refine the data selected.

![Outlier_fig1][2]

At this point, you may want to look directly at the HST data. This can be done by clicking on the  icon (<span style="color:yellow;">yellow</span>) under Actions, and then clicking on the Preview image (see Use Case 1 for details).

Note that while the HSC can be used directly for some science projects, in many cases (like this) you would need to make a much deeper catalog to get the most out of the data. In these cases the HSC provides: 1) a quick look at what data exists, 2) a feasibility study, 3) a dataset to compare with at the brighter wavelengths, and 4) the ability to link with other instruments or surveys.


  [1]: http://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html
  [2]: screenshots/Outlier_fig1.png

## <span style="color:red;">Step 3</span> - Make a Color-Magnitude Diagram (CMD)

Make a plot by clicking the ![plot][1] icon (<span style="color:blue;">blue</span>). Under Configuration, make an U-I (W3_F275W-W3_F814W) column by using the Combine Columns option (<span style="color:green;">green</span>).

![Outlier_fig2][2]

Next, select X = U-I and Y = W3_F275W (<span style="color:blue;">blue</span>). Click on the Update Series button (<span style="color:green;">green</span>) to plot the CMD. Reverse the Y axis to have magnitudes go from bright to faint (<span style="color:orange;">orange</span>). The white dwarfs are blueward of the main band of stars (<span style="color:yellow;">yellow</span>).

![Outlier_fig3][3]

In some cases, you may find that there are objects in the CMD where you don't expect them. If you look at the details for these objects, you may find that they have particular high CI values (possibly due to a cosmic ray or other defect in the image) or large magnitude errors (due to the inclusion of a saturated measurement). While we attempt to screen these out when building the source lists, we are not always successful. In these cases, you should use the Filtering options to screen out the bad data.



  [1]: screenshots/plot.png
  [2]: screenshots/Outlier_fig2.png
  [3]: screenshots/Outlier_fig3.png

## <span style="color:red;">Step 4</span> - Expand the dataset

The Discovery Portal found 50,000 objects, which is currently the limit of the number of objects it will report per search. Many of these do not have F275W and F814W magnitudes, so we may be missing (many) other objects that would be of interest. Also, we have restricted our search to only 200" when ideally we would want to cover the entire cluster, which would result in even more objects that would not be reported due to the 50,000 object limit. To do a more complete search, we will use CasJobs (see Use Case 2 for more details on using CasJobs).

![Outlier_fig4][1]

Enter the [HSC CasJobs][2] webpage and login. Go to the Query page (<span style="color:blue;">blue</span>) and set the context to HSC (<span style="color:green;">green</span>). The query below is designed to find objects in M4 that have both WFC3 F275W and F814W magnitudes. Note that we have expanded the search radius to 500s to encompass the entire cluster, and restricted the search to exclude objects with high CI values and large magnitude errors.

```sql
SELECT
   MatchRA, MatchDEC, MatchID, W3_F275W, W3_F814W,
   F275W_F814W=W3_F275W-W3_F814W
FROM 
  SearchSumCatalog(245.89750,-26.52553,500.0,1)
  where CI < 1.25
  and numimages > 2
  and (W3_F275W > 0 and W3_F814W > 0) 
  and W3_F275W_Sigma < 1.0
  and W3_F814W_Sigma < 1.0
ORDER BY matchID
INTO M4
```

The WHERE clause defines the search parameters:
     i) concentration index (CI) is less than 1.25 (to find only stars)
     ii) number of images in a match (>2)
     iii) W3_F275W and W3_F814W must be greater than 0 (to ignore objects with only one color)
     iv) W3_F275W_Sigma and W3_F814W_Sigma must be less than 1.0 (to ignore objects with poor measurements)

Submit (<span style="color:orange;">orange</span>) the query for processing.


  [1]: screenshots/Outlier_fig4.png
  [2]: http://mastweb.stsci.edu/hcasjobs/

## <span style="color:red;">Step 5</span> - Export the CasJobs data

After the query has finished, go to the MyDB (<span style="color:blue;">blue</span>) page. Find the dataset (M4) in the list on the left and select it (<span style="color:green;">green</span>). Note that there are ~381 objects. We need to download the data so we can import it into the Discovery Portal for plotting. Select the Download option (<span style="color:orange;">orange</span>). Select your format (<span style="color:yellow;">yellow</span>) and click on Go.

![Outlier_fig5][1]

After hitting Go, you are taken to the Output page. Once the dataset is available, click on Download (<span style="color:blue;">blue</span>).

![Outlier_fig6][2]


  [1]: screenshots/Outlier_fig5.png
  [2]: screenshots/Outlier_fig6.png

## <span style="color:red;">Step 6</span> - Import the catalog into the Discovery Portal

Prior to ingesting the catalog into the Discovery Portal, you must change the name of the MatchRA column to RA and the MatchDEC column to DEC; you can also rename the F275W-F814W column to U-I. Then go to the Discovery Portal and click on the ![import][1] icon (<span style="color:blue;">blue</span>) to ingest the catalog. To see the file format, click on the ![help][2] icon (<span style="color:green;">green</span>). After browsing to input the file name, click on Upload (<span style="color:orange;">orange</span>) to ingest the catalog.

![Outlier_fig7][3]

After importing, you will see the catalog and the objects overlaid on the DSS image.


  [1]: screenshots/import.png
  [2]: screenshots/help.png
  [3]: screenshots/Outlier_fig7.png

## <span style="color:red;">Step 7</span> - Make the Color-Magnitude Diagram

As we did in Step 3, make the plot by clicking the ![plot][1] icon. Note that we do not need to construct the U-I color as we did in Step 2 as we did that when we creating the catalog. Under Configuration, select X = U-I and Y = W3_F275W. Click on the Update Series button to plot the CMD. Reverse the Y axis to have magnitudes go from bright to faint. Note that there are many more objects in the plot than we had using the Discovery Portal query.

![Outlier_fig8][2]

The white dwarfs are blueward of the main band of stars. Of the 8 objects in the WD area, 6 are in Richer, 1 is not in the WFPC2 FOV (<span style="color:green;">green</span>) used by Richer and hence is a new white dwarf candidate, and 1 is not visible on the WFPC2 F814W image (but is in the WFC3 data so is in the HSC; <span style="color:orange;">orange</span>).

It is always a good idea to go back to the HLA and look at each object to make certain it is not an artifact or have other issues. Below is a blowup of the one candidate (<span style="color:blue;">blue</span>) that was not visible on the original WFPC2 F814W image.

![Outlier_fig9][3]

With the techniques above, it is easy to make other Color-Magnitude diagrams.

![Outlier_fig10][4]


  [1]: screenshots/plot.png
  [2]: screenshots/Outlier_fig8.png
  [3]: screenshots/Outlier_fig9.png
  [4]: screenshots/Outlier_fig10.png